## Problem 3: How many people live near shopping centers? (8 points)

In the last step of this analysis, use a *spatial join* to relate data from a population grid data set to the buffer layer created in *problem 2* to find out how many people live in all population grid cells that are **within** 1.5 km distance from each shopping centre. 

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS).


*Feel free to divide your solution into more codeblocks than prepared! Remember to add comments to your code :)*

### a) Load the population grid data set and the buffer geometries (2 points)

Use the same population grid data set as during [lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/spatial-join.html) (load it directly from WFS, don’t forget to assign a CRS). Load the data into a `GeoDataFrame` called `population_grid`.

(optional) If you want, discard unneeded columns and translate the remaining column names from Finnish to English.

In [8]:
import geopandas
import pathlib 
NOTEBOOK_PATH = pathlib.Path().resolve()
population_grid = geopandas.read_file(NOTEBOOK_PATH / 'Vaestotietoruudukko_2020.xml')
population_grid.crs = 'EPSG:3879'

In [9]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
import geopandas
import pyproj

assert isinstance(population_grid, geopandas.GeoDataFrame)
assert population_grid.crs == pyproj.CRS('EPSG:3879')



Load the buffers computed in *problem 2* into a `GeoDataFrame` called `shopping_centre_buffers`. Add an `assert` statement to check whether the two data frames are in the same CRS.

In [17]:
shopping_centre_buffers = geopandas.read_file(NOTEBOOK_PATH / 'shopping_centres.gpkg', layer='buffers')
assert population_grid.crs == pyproj.CRS('EPSG:3879')
assert shopping_centre_buffers.crs == pyproj.CRS('EPSG:3879')
assert population_grid.crs == shopping_centre_buffers.crs


In [18]:
# NON-EDITABLE CODE CELL FOR TESTING YOUR SOLUTION
assert isinstance(shopping_centre_buffers, geopandas.GeoDataFrame)
assert shopping_centre_buffers.geometry.geom_type.unique() == ["Polygon"]
assert shopping_centre_buffers.crs == pyproj.CRS("EPSG:3879")


---

### b) Carry out a *spatial join* between the `population_grid` and the `shopping_centre_buffers`  (2 points)

Join the shopping centre’s `id` column (and others, if you want) to the population grid data frame, for all population grid cells that are **within** the buffer area of each shopping centre. [Use a *join-type* that retains only rows from both input data frames for which the geometric predicate is true](https://geopandas.org/en/stable/gallery/spatial_joins.html#Types-of-spatial-joins). 


In [21]:
joint_data = geopandas.sjoin(population_grid, shopping_centre_buffers, how='inner', predicate='within')
joint_data.head()


,gml_id,index,asukkaita,asvaljyys,ika0_9,ika10_19,ika20_29,ika30_39,ika40_49,ika50_59,ika60_69,ika70_79,ika_yli80,geometry,index_right,id,name,addr,address
1147,Vaestotietoruudukko_2020.1148,8371,128,52,15,25,4,7,32,16,19,9,1,"POLYGON ((25484250.000 6672499.005, 25484250.0...",2,3,Iso Omena,"Piispansilta 11, 02230 Espoo, Finland","Musti ja Mirri, 11, Piispansilta, Matinkylä, S..."
1148,Vaestotietoruudukko_2020.1149,8372,81,57,99,99,99,99,99,99,99,99,99,"POLYGON ((25484250.000 6672249.006, 25484250.0...",2,3,Iso Omena,"Piispansilta 11, 02230 Espoo, Finland","Musti ja Mirri, 11, Piispansilta, Matinkylä, S..."
1149,Vaestotietoruudukko_2020.1150,8374,20,61,99,99,99,99,99,99,99,99,99,"POLYGON ((25484250.000 6671748.997, 25484250.0...",2,3,Iso Omena,"Piispansilta 11, 02230 Espoo, Finland","Musti ja Mirri, 11, Piispansilta, Matinkylä, S..."
1211,Vaestotietoruudukko_2020.1212,8532,110,50,14,20,2,10,15,24,7,15,3,"POLYGON ((25484499.998 6672749.004, 25484499.9...",2,3,Iso Omena,"Piispansilta 11, 02230 Espoo, Finland","Musti ja Mirri, 11, Piispansilta, Matinkylä, S..."
1212,Vaestotietoruudukko_2020.1213,8533,136,53,18,28,7,10,25,21,15,11,1,"POLYGON ((25484499.998 6672499.005, 25484499.9...",2,3,Iso Omena,"Piispansilta 11, 02230 Espoo, Finland","Musti ja Mirri, 11, Piispansilta, Matinkylä, S..."



---

### c) Compute the population sum around shopping centres (2 points)

Group the resulting (joint) data frame by shopping centre (`id` or `name`), and calculate the `sum()` of the population living inside the 1.5 km radius around them.

Print the results, for instance, in the form "12345 people live within 1.5 km from REDI".

In [22]:
population_sum = joint_data.groupby('id')['asukkaita'].sum().reset_index()
for _, row in population_sum.iterrows():
    shopping_centre_name = shopping_centre_buffers[shopping_centre_buffers['id'] == row['id']]['name'].values[0]
    print(f"{row['asukkaita']} people live within 1.5 km from {shopping_centre_name}")


19113 people live within 1.5 km from Itis
55357 people live within 1.5 km from Forum
26848 people live within 1.5 km from Iso Omena
23072 people live within 1.5 km from Sello
10956 people live within 1.5 km from Jumbo
21842 people live within 1.5 km from REDI
24394 people live within 1.5 km from Tripla



---

### d) Reflection

Good job! You are almost done with this week’s exercise. Please quickly answer the following short questions:
    
- How challenging did you find problems 1-3 (on scale to 1-5), and why?
- What was easy?
- What was difficult?

Add your answers in a new *Markdown* cell below:

I rate the difficulty of these problems as 2 out of 5. Most of the tasks were straightforward and short but also quite interesting. 
The only issue I encountered was with problem 3(a) related to fetching the grid population data. 
The method shown in lesson 3 (gpd.read_file(url)) did not work for me. 
I attempted a workaround using owslib but still couldn't access the WFS. 
I resolved the issue by manually downloading the required file.